# Cross Beam Energy Transfer
What is the process for CBET?

## Yorick Script
1. Initialization
    1. Domain (x, z, dx, dz)
    2. Physics ($n_e$, $T_e$, $\lambda$, $\omega_{pe}$,...)
    3. Beam profile (power, phase,...)
2. Beam Initialization
    1. Initial ray weighting
        1. `uray[1] = (I0 * CFL / rays_per_zone) * interpolate(pow_x, phase_x + offset, z0)`
    2. Increment total energy
        1. `injected += uray[1]`
    3. Launch Ray
        1. Find logical indices of ray initial position. (thisx_0, thisx_00,...)
        2. Calclate total $k$ from dispersion relation. $k = \sqrt{\frac{\omega^2 - \omega_{pe}^2}{c^2}}$
        3. Normalize $k$
        4. Initialize ray $k$ vector
            1. $k_{ray} = \frac{k_{init}}{|k|} k$
        5. Initialize ray group velocity $\frac{\partial \omega}{\partial k}$
            1. $v_{ray} = \frac{c^2 k_{ray}}{\omega}$
        6. Time stepping loop
            1. Update group velocities using $\nabla n_e$
                1. $v_g = \frac{c^2}{2 n_c}\nabla n_e \delta t$
            2. Update position (for timestep n)
                1. $x^n = x^{n-1} + v_x^n \delta t$
                2. $z^n = z^{n-1} + v_z^n \delta t$
            3. Update position
            4. Find and mark all cells ray travels through.
            5. Calculate resistivity $\eta$ and collision frequency $\nu_{ei}$
            6. Update rays energy by amount absorbed by plasma
                1. $u_{ray}^n = u_{ray}^{n-1}\left(1 - \frac{n_e}{n_c}\nu_{ei} \delta t \right)$
            7. Deposit energy into nearest neighboring cells
                1. Determine distance to nearest node $(x_p, z_p)$
                2. Weight deposition based on distance to nearest 4 nodes (8 in 3D)
            8. Update electric field amplitude
                1. $\left(\frac{\omega^2 - \omega_{pe,x_0,z_0}^2}{\omega^2 - \omega_{pe,x,z}^2}\right)^{1/4}$
            9. Repeat until ray exits domain. 
3. Beam Intersections
    1. Check all zones for intersections between beams
4. Calculate CBET interactions, gain, and iterate solution.
    1. Probe beam gains energy when gain < 0, loses energy when gain > 0
    2. Pump beam loses energy when gain < 0, gains energy when gain > 0
    3. For every ray intersection
        1. Calculate new $k$ vectors of each ray
        2. Calculate resistivity of plasma
            1. $\eta = \frac{(\omega_2 - \omega_1) - (k_{x2} - k_{x1})u_{flow}}{\omega_s}$
            2. $\omega_s = \frac{k_{iaw}}{C_s}$, acoustic frequency
            3. $u_{flow} = \frac{\text{mach number}}{C_s}$, plasma flow velocity
            4. $C_s$, acoustic wave speed
        3. Calculate initial electric field of ray
            1. $E_b = \sqrt{\frac{8\pi\times 10^7 I_{b}}{c}}$
        4. Calculate power
            1. $P = \frac{\eta \nu_{iaw}^2}{(\eta^2 - 1)^2 + (\eta\nu_{iaw})^2}$
            2. $\nu_{iaw}$, ion-acoustic wave energy damping rate $\frac{\nu_{ia}}{\omega_s}$?
        5. Calculate gain
            1. $G_1 = \alpha_1 E_2^2 P \frac{n_e}{n_c \nu_{iaw}}$
            2. $G_2 = \alpha_1 E_1^2 P \frac{n_e}{n_c \nu_{iaw}}$
            3. $\alpha_1 = \frac{e^2}{4m_e c \omega K_b T_e (1 + \frac{3T_i}{Z T_e})}$, $e$ is charge in statcoulombs, $m_e$ is electron mass in grams
        6. Update energy
            1. $W_2 = W_2 \exp\left[ - \frac{W_1 |k| G_2}{\sqrt{1 - \frac{c_e}{c_c}}} \right]$
            2. $W_1 = W_1 \exp\left[ - \frac{W_2 |k| G_1}{\sqrt{1 - \frac{c_e}{c_c}}} \right]$
5. Update Intensities
    1. For every ray intersection
        1. $\Delta I_1 = -I_{b1} \left(1 - \frac{W_{1}}{W_{1,0}} \right)$
        2. $\Delta I_2 = -I_{b2} \left(1 - \frac{W_{2}}{W_{2,0}} \right)$
        3. $I_{ray} = I_{ray} + \Delta I_{ray}$
        4. Adjust every other ray in current cell
        

            


## Alex CUDA CBET

1. Read in $n_e$ and $T_e$ data.
2. Initialize memory for arrays.
    1. init `marked` to -1
    2. init `kvec` to 0.13 (Omega) or 5E-4 (other)
    3. init `polar_angle` to $\pi/2$
    4. init `eden` by interpolating $n_e$ from spherical to cartesian
3. Start ray tracing (call `rayTracing()`)
    1. init `phase_r` and `pow_r` 
    2. set `dbeam_normal` to `beam_normal`
        1. Why is this necessary?
    3. Launch kernel `launch_ray_XYZ`
        1. Iterate over all rays in beam
            1. initialize ray
                1. I don't understand what the `child` values are for...
                2. Interpolate of x, y, z domain.
                3. Rotate ray around beam normal
                4. Interpolate ray power
                5. why use `__any_sync()`??
            2. Store initial ray power
            3. Store initial k vectors
            4. Calculate total k (called `parent_w`)
            5. Calculate velocity components
                1. Sets parent/child velocity to negative beam normal.
                2. $v_{parent} = -\vec{n}_{beam} \frac{c^2 k_{parent}}{k_{norm} \omega}$
                3. Repeat for `child1` and `child2`
                4. Convert cartesian velocity to polar coordinates
            6. Time Stepping Loop
                1. `timestep` is called three times, with first argument 0, 1, 2. If `r == 0`, the `polar_angle` array is updated. No difference for `r == 1` or `r == 2` except `child` values are used.
                    1. Set bounds on x,y,z $(x_m, x_p)$, $(y_m, y_p)$, $(z_m, z_p)$
                    2. Load $n_e$ into local variables
                    3. Calculate change in $k$
                        1. $\delta \vec{k} = \frac{c^2 \delta t}{4 n_c \Delta} (n_{e, p} - n_{e, m})$
                        2. $\delta \vec{k}$ is the negative change of $\vec{k}$
                    4. Compute polarization
                        1. $\delta \vec{P} = -\vec{v} \frac{\vec{x}_{polar} \cdot \delta}{|k|^2}$
                        2. $\vec{x}_{polar} += \delta \vec{P}$   
                        3. $\vec{x}_{polar} = \frac{\vec{x}_{polar}}{|\vec{x}_{polar}|}$
                    5. Update velocity
                        1. $\vec{v} = \vec{V} - \delta \vec{k}$
                    6. Update `space_x`/`space_y`/`space_z`
                        1. $\vec{x} = \vec{x} + \vec{v}\delta t$
                    7. Project $\vec{v}$ onto polar plane and normalize
                        1. Why?
                    8. Update `current_phase`
                        1. Declared and set to 0 in `launch_ray_XYZ`
                        2. Never used or returned.
                    9. Determine current `x/y/z` indices
                    10. Check if ray intensity is high enough and ray has moved to new cell
                        1. Why would ray intensity be too low? `uray >  1E-12`
                        2. Update `k_vec` with $\vec{x}$
                        3. `if r == 0` convert polar coordinates to angle, store in `polar_angle` array.
                        4. increment `num_crossings`
                    11. Kill ray if it leaves domain
                    12. `return __all_sync(0xFFFFFFFF, kill)`
                        1. returns 0 if any ray was killed
                2. If ray is killed, exit time stepping.
    4. Assert ray angles are "stable"
        1. What does this mean?
    5. Launch kernel `compute_area_sheets`
        1. Iterate over all rays in beam
            1. if parent k vector is greater than space size, continue
            2. Set lengths to 0
            3. Set `j` and `k` to 1
            4. load parent and children k vectors
                1. Does `k_parent` hold k vector or position? Are they the same thing?
            5. Calculate trianglular area with parent/children positions as vertices.
                1. Is this a poor-mans bezier curve?
            6. Iterate over number of crossings
                1. Accumulate normalized difference between current and previous positions (`k_parent`)
                2. if `k_parent_x > spacesize` exit loop
                3. For `k_child1` and `k_child2` iterate over remaining crossings
                    1. exit if `k_child > 1.75 * spacesize`
                        1. Why 1.75 * spacesize?
                    2. Calculate normalized difference of `k_child` with previous time step.
                    3. If childs length is greater than parents length
                        1. accumulate normalized child differences
                        2. set `child_equal` to difference of child and parent divided by normalized difference
                            1. `child1_equal = (parent_length - child_length) / norm(k_child[j] - k_child[j - 1])
                        3. exit loop
                    4. update normalized difference and loop
                4. Update child positions?
                5. Increment `j` and `k`
                6. Calculate new trianglar area
                7. Update parent/children x/y/z by subtracting previous values, set previous values to current values
                8. Update `areas` array with ratio of new area to initial area
                9. Calculate x/y/z indices
                10. Calculate maximum amplitude
                    1. `max_amp = 1 / (area * parent_local_eps)`
                    2. `parent_local_eps = 1 - (eden[parent_idx] / ncrit)`
        2. Set maximum amplitude index array to max found amplitude.
    6. Launch kernel `fill_marked`
        1. Iterate over all rays
            1. Iterate over all crossings
                1. break loop if `space_x > spacesize`
                2. Find x/y/z indices
                3. Increment `counter` by 1 at given indices (AtomicAdd)
                4. `count` is `counter[i*ny*nz + j*nz + k]` old value (before AtomicAdd)
                5. Swaps `raynum` with current value of `marked[count*nx*ny*nz + i*ny*nz + j*nz + k]`
                    1. What does this accomplish?
4. Interpolate `machnum` onto 3D cartesian
5. Iterate over all beams, rays, crossings and accumulate total intensities.
6. Start CBET process (call `cbet()`)
    1. Launch kernel `polarization_update`
        1. Iterate over all rays
            1. Skip ray if previous x is greater than spacesize
            2. Iterate over all crossings
                1. Exit crossing loop if x is greater than spacesize
                2. Get spatial indices
                3. Restrict search space for node_x/y/z
                4. Calculate bunch of parameters
                5. Calculate electric field components
                6. Calculate density gradient
                7. Set electric field amplitudes
    2. Do-while loop until convergence (ugh...)
        1. Increment `iteration` and `sense_rev` counters
            1. What is `sense_rev`???
        2. Launch kernel `cbet_gain`
            1. Couple exit conditions at beginning
            2. Iterate over crossings 
                1. Load k-vectors
                2. Get spatial indices
                3. Difference k-vectors
                    1. ???
                4. Normalize to k-vectors to step size
                5. Iterate over rays in beam
                    1. Calculate ray and sheet indices
                    2. Iterate over sheets
                        1. Skip sheet `if sheet == beam`
                            1. ????
                        2. Skip sheet if `ray_o == -1`
                        3. Load k-vectors
                        4. Calculate indices
                        5. Increment `raycross` in while loop
                            1. number of crossings until... original indices?
                        6. Calculate k-vector of pump ray, normalize?
                        7. Calculate mach numbers
                        8. Caculate ion-acoustic wave frequency and vector
                        9. Calculate $\eta$
                            1. resistivity?
                        10. Calculate coupling multipliers
                        11. Calculate electric field components
                        12. Sum electric field components time coupling multipliers
                    3. Store summed electric fields
        3. Sum over `wMult` two times
            1. frequency multiplier?
        4. Launch kernel `update_i_cu`
            1. Load electric field amplitudes and `wMult`
            2. Exit if $E_x$ is too small
            3. Iterate over crossings
                1. Accumulate `wMult` multipliers
                2. Exit loop if previous $E_x$ value is zero
                3. Calculate absorption coefficient
                    1. ????
                4. Calculate fractional change in intensity
                5. Store max fractional change
                6. Update electric field with fractional change
                    1. `local_max` is set to `max(local_max, frac_x, frac_y)` 
                    2. `local_max < frac_x/y`
                    2. If `frac_x` or `frac_y` is the local max, nothing gets updated
                7. Store electric field values
                8. Update intensity array
            4. Check if `local_max > convergenceMax`
                1. Do-While `current != var` and `local_max > current` 
                    1. Store current convergence maximum
                    2. AtomicCAS local_max and convergenceMax
        5. Resets `sense_rev` every 1000 steps
        6. Increments convergence condition value
            1. ???? 
        7. Breaks Do-While loop unconditionally
            1. Does the Do-While ever run more than once?
7. Iterate over all beam, rays, and crossings
    1. Update `intenmap` and `total_inten`
8. Free CUDA memory `i_b, machnum, wMult, kvec, intensity_init`
9. Iterate over all X/Y/Z
    1. accumulate `intenmap` into itself
10. Free CUDA memory `marked`
11. Program exits